# Doc Q&A Demo
This notebook contains an example of Doc Q&A, where a user can upload a document and ask questions about it. The pipeline will take the following steps:
1. Load text documents
2. Chunk the text
3. Embed each chunk
4. Index chunks and store in an in-memory vector database
5. Retrieve relevant chunks given a user query

The example is built with the langchain library.

References: 
* https://python.langchain.com/docs/integrations/vectorstores/faiss/
* https://huggingface.co/learn/cookbook/advanced_rag

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

# Load text documents

Using `TextLoader`to load a single text document (https://python.langchain.com/docs/modules/data_connection/document_loaders/).

To load all files within a directory, use `DirectoryLoader` (https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory/)

In [17]:
document_path = "docs/state_of_the_union.txt"
loader = TextLoader(document_path)
document= loader.load()

In [21]:
print(type(document), len(document))
doc_0 = document[0]
print(type(doc_0))

<class 'list'> 1
<class 'langchain_core.documents.base.Document'>


In [23]:
print(dir(doc_0))

['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict_if_root', '_get_value', '_init_private_attribute

In [28]:
print(type(doc_0.page_content))
doc_0.page_content[:150]

<class 'str'>


'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fello'

# Split the document



In [29]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(document)

In [32]:
type(docs), len(docs)

(list, 42)

In [ ]:



#embeddings = OpenAIEmbeddings()
#db = FAISS.from_documents(docs, embeddings)
#print(db.index.ntotal)